# FathomNet

Training PROB on custom data.

In [1]:
data_name = "fathomnet" # give your dataset a name

In [2]:
# # run this to create the necessary folder
# import os 

# data_dir = "./data/OWOD/"
# folders = ['JPEGImages', 'Annotations', 'ImageSets']

# for folder in folders:
#     try:
#         os.makedirs(os.path.join(data_dir, folder, data_name))
#     except OSError as e:
#         print(f"Can't create folder: {str(e)}")

Let's say you want to add a dataset "DATASET_A". Then you need to:

1. In `./datasets/torchvision_datasets/open_world.py` line 120, add to the dictionary VOC_COCO_CLASS_NAMES a key-value pair: VOC_COCO_CLASS_NAMES["DATASET_A"]=["a","b","c",...]
2. Store DATASET_A's images under "data/OWOD/JPEGImages/DATASET_A/"
3. Store DATASET_A's Annotations under "data/OWOD/Annotations/DATASET_A/"
4. Store DATASET_A's ImageSets files under "data/OWOD/ImageSets/DATASET_A/"
5. When you train, the input --dataset should be set to DATASET_A (e.g., --dataset DATASET_A)

In [3]:
import itertools

UNK_CLASS = ["unknown"]

VOC_COCO_CLASS_NAMES = {}

T1_CLASS_NAMES = [
    'Urchin', 'Fish', 'Sea star', 'Anemone', 'Sea cucumber', 
    'Sea pen', 'Sea fan', 'Worm', 'Crab', 'Gastropod'
]

T2_CLASS_NAMES = [
    'Shrimp', 'Soft coral'
]

T3_CLASS_NAMES = [
    'Glass sponge', 'Feather star'
]

T4_CLASS_NAMES = [
    'Eel', 'Squat lobster', 'Barnacle', 'Stony coral', 'Black coral', 'Sea spider'
]

VOC_COCO_CLASS_NAMES["fathomnet"] = tuple(itertools.chain(T1_CLASS_NAMES, T2_CLASS_NAMES, T3_CLASS_NAMES, T4_CLASS_NAMES, UNK_CLASS))


In [4]:
VOC_COCO_CLASS_NAMES

{'fathomnet': ('Urchin',
  'Fish',
  'Sea star',
  'Anemone',
  'Sea cucumber',
  'Sea pen',
  'Sea fan',
  'Worm',
  'Crab',
  'Gastropod',
  'Shrimp',
  'Soft coral',
  'Glass sponge',
  'Feather star',
  'Eel',
  'Squat lobster',
  'Barnacle',
  'Stony coral',
  'Black coral',
  'Sea spider',
  'unknown')}

Other files to change:

- `configs/M_OWOD_BENCHMARK.sh` update all paths to point to the correct ImageSet files
- `/home/sabrina/code/PROB/run.sh` update the number of GPUs you have in your machine

Sort out WANDB:
- change entity (aka wandb username) in lines 165 and 167 in the file `/main_open_world.py`

In [5]:
import wandb

# confirm login
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: spereira. Use `wandb login --relogin` to force relogin


True

There are issues with the file names, they have to be numbers like the VOC dataset for some reason. I made a VOC-test folder with all the file sthat need to be changed.
- Annotations and JPEGImages need a new file name, and 
- all txt files inside ImageSets need to be updated to match.

If it all goes wrong and all files need to be copied again do it in bash in the `./data/processed` folder `cp -frp VOC-backup -T VOC-test` make sure the taget folder name doesn't exist already ([stackoverflow](https://stackoverflow.com/questions/33343840/bash-duplicate-rename-folder)).

In `datasets/torchvision_datasets/open_world.py` lines 193 and 198, change .jpg to .png as our images are pngs.

## Set up

### Compiling CUDA operators

In [6]:
# %cd models
# !wget https://dl.fbaipublicfiles.com/dino/dino_resnet50_pretrain/dino_resnet50_pretrain.pth
# %cd ops
# !sh ./make.sh
# %cd ../..

## Inference

Orr's [reply](https://github.com/orrzohar/PROB/issues/34).

Notebooks for ref: 
- [Objective: fine-tuning DETR](https://github.com/woctezuma/finetune-detr/blob/master/finetune_detr.ipynb)
- [Object Detection with DETR - a minimal implementation](https://colab.research.google.com/github/facebookresearch/detr/blob/colab/notebooks/detr_demo.ipynb#scrollTo=kqe_0nc5dyAq)

Questions:
- how do i load the model from the pre-trained weights?

In [1]:
# from models.prob_deformable_detr import build
from models.deformable_detr import build
from models import build_model
import numpy as np
import torch

/home/sabrina/mambaforge/envs/prob2/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class Args:
    lr = 2e-4
    lr_backbone_names = ["backbone.0"]
    lr_backbone = 2e-5
    lr_linear_proj_names = ['reference_points', 'sampling_offsets']
    lr_linear_proj_mult = 0.1
    batch_size = 5
    weight_decay = 1e-4
    epochs = 51
    lr_drop = 35
    lr_drop_epochs = None
    clip_max_norm = 0.1
    sgd = False
    with_box_refine = False
    two_stage = False
    masks = False
    backbone = 'dino_resnet50'
    frozen_weights = None
    dilation = False
    position_embedding = 'sine'
    position_embedding_scale = 2 * np.pi
    num_feature_levels = 4
    enc_layers = 6
    dec_layers = 6
    dim_feedforward = 1024
    hidden_dim = 256
    dropout = 0.1
    nheads = 8
    num_queries = 100
    dec_n_points = 4
    enc_n_points = 4
    aux_loss = True
    set_cost_class = 2
    set_cost_bbox = 5
    set_cost_giou = 2
    cls_loss_coef = 2
    bbox_loss_coef = 5
    giou_loss_coef = 2
    focal_alpha = 0.25
    coco_panoptic_path = None
    remove_difficult = False
    output_dir = ''
    device = 'cuda'
    seed = 42
    resume = './exps/MOWODB/PROB/t1/checkpoint0040.pth'
    start_epoch = 0
    eval = True
    viz = False
    eval_every = 5
    num_workers = 3
    cache_mode = False
    PREV_INTRODUCED_CLS = 0
    CUR_INTRODUCED_CLS = 10
    unmatched_boxes = False
    top_unk = 5
    featdim = 1024
    invalid_cls_logits = False
    NC_branch = False
    bbox_thresh = 0.3
    pretrain = './exps/MOWODB/PROB/t1/checkpoint0039.pth'
    nc_loss_coef = 2
    train_set = ''
    test_set = ''
    num_classes = 21
    nc_epoch = 0
    dataset = 'fathomnet'
    data_root = '/home/sabrina/code/PROB/data/OWOD'
    unk_conf_w = 1.0
    model_type = 'prob'
    wandb_name = ''
    wandb_project = 'fathomnet'
    obj_loss_coef = 1
    obj_temp = 1
    freeze_prob_model = False
    num_inst_per_class = 50
    exemplar_replay_selection = False
    exemplar_replay_max_length = 1e10
    exemplar_replay_dir = ''
    exemplar_replay_prev_file = ''
    exemplar_replay_cur_file = ''
    exemplar_replay_random = False

args = Args()

In [3]:
model, criterion, postprocessors, exemplar_selection = build_model(args, mode=args.model_type)

Invalid class range: [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
DINO resnet50


/home/sabrina/mambaforge/envs/prob2/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/sabrina/mambaforge/envs/prob2/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


running with exemplar_replay_selection


In [4]:
device = torch.device(args.device)

model.to(device)

model_without_ddp = model

def match_name_keywords(n, name_keywords):
    out = False
    for b in name_keywords:
        if b in n:
            out = True
            break
    return out

param_dicts = [
    {
        "params":
            [p for n, p in model_without_ddp.named_parameters()
                if not match_name_keywords(n, args.lr_backbone_names) and not match_name_keywords(n, args.lr_linear_proj_names) and p.requires_grad],
        "lr": args.lr,
    },
    {
        "params": [p for n, p in model_without_ddp.named_parameters() if match_name_keywords(n, args.lr_backbone_names) and p.requires_grad],
        "lr": args.lr_backbone,
    },
    {
        "params": [p for n, p in model_without_ddp.named_parameters() if match_name_keywords(n, args.lr_linear_proj_names) and p.requires_grad],
        "lr": args.lr * args.lr_linear_proj_mult,
    }
]

if args.sgd:
    optimizer = torch.optim.SGD(param_dicts, lr=args.lr, momentum=0.9,
                                weight_decay=args.weight_decay)
else:
    optimizer = torch.optim.AdamW(param_dicts, lr=args.lr,
                                    weight_decay=args.weight_decay)

In [5]:
if args.pretrain:
    print('Initialized from the pre-training model')
    checkpoint = torch.load(args.pretrain, map_location='cpu')
    state_dict = checkpoint['model']
    msg = model_without_ddp.load_state_dict(state_dict, strict=False)
    print(msg)
    args.start_epoch = checkpoint['epoch'] + 1
    if args.eval:
        test_stats, coco_evaluator = evaluate(model, criterion, postprocessors, data_loader_val, base_ds, device, args.output_dir, args)
        #return
        

Initialized from the pre-training model
<All keys matched successfully>


NameError: name 'evaluate' is not defined

: 

In [11]:
    if args.resume.startswith('https'):
        checkpoint = torch.hub.load_state_dict_from_url(
            args.resume, map_location='cpu', check_hash=True)
    else:
        checkpoint = torch.load(args.resume, map_location='cpu')
    missing_keys, unexpected_keys = model_without_ddp.load_state_dict(checkpoint['model'], strict=False)
    unexpected_keys = [k for k in unexpected_keys if not (k.endswith('total_params') or k.endswith('total_ops'))]
    if len(missing_keys) > 0:
        print('Missing Keys: {}'.format(missing_keys))
    if len(unexpected_keys) > 0:
        print('Unexpected Keys: {}'.format(unexpected_keys))
    # if not args.eval and 'optimizer' in checkpoint and 'lr_scheduler' in checkpoint and 'epoch' in checkpoint:
    #     import copy
    #     p_groups = copy.deepcopy(optimizer.param_groups)
    #     optimizer.load_state_dict(checkpoint['optimizer'])
    #     for pg, pg_old in zip(optimizer.param_groups, p_groups):
    #         pg['lr'] = pg_old['lr']
    #         pg['initial_lr'] = pg_old['initial_lr']
    #     print(optimizer.param_groups)
    #     lr_scheduler.load_state_dict(checkpoint['lr_scheduler'])
    #     # todo: this is a hack for doing experiment that resume from checkpoint and also modify lr scheduler (e.g., decrease lr in advance).
    #     args.override_resumed_lr_drop = True
    #     if args.override_resumed_lr_drop:
    #         print('Warning: (hack) args.override_resumed_lr_drop is set to True, so args.lr_drop would override lr_drop in resumed lr_scheduler.')
    #         lr_scheduler.step_size = args.lr_drop
    #         lr_scheduler.base_lrs = list(map(lambda group: group['initial_lr'], optimizer.param_groups))
    #     lr_scheduler.step(lr_scheduler.last_epoch)
    #     args.start_epoch = checkpoint['epoch'] + 1
    # check the resumed model
    # if (not args.eval and not args.viz and args.dataset in ['coco', 'voc']):
    #     test_stats, coco_evaluator = evaluate(
    #         model, criterion, postprocessors, data_loader_val, base_ds, device, args.output_dir, args
    #     )
    # if args.eval:
    #     test_stats, coco_evaluator = evaluate(model, criterion, postprocessors, data_loader_val, base_ds, device, args.output_dir, args)
    #     if args.output_dir:
    #         utils.save_on_master(coco_evaluator.coco_eval["bbox"].eval, output_dir / "eval.pth")
    #     return

RuntimeError: Error(s) in loading state_dict for DeformableDETR:
	size mismatch for query_embed.weight: copying a param with shape torch.Size([100, 256]) from checkpoint, the shape in current model is torch.Size([100, 512]).

In [33]:
state_dict = torch.load(open('./exps/MOWODB/PROB/t1/checkpoint0040.pth', 'rb'), map_location='cpu')

# for i in state_dict['model'].keys():
#     print(i)

In [37]:
missing_keys, unexpected_keys = model.load_state_dict(state_dict['model'], strict=False)
#model.eval()

RuntimeError: Error(s) in loading state_dict for DeformableDETR:
	size mismatch for transformer.encoder.layers.0.linear1.weight: copying a param with shape torch.Size([2048, 256]) from checkpoint, the shape in current model is torch.Size([1024, 256]).
	size mismatch for transformer.encoder.layers.0.linear1.bias: copying a param with shape torch.Size([2048]) from checkpoint, the shape in current model is torch.Size([1024]).
	size mismatch for transformer.encoder.layers.0.linear2.weight: copying a param with shape torch.Size([256, 2048]) from checkpoint, the shape in current model is torch.Size([256, 1024]).
	size mismatch for transformer.encoder.layers.1.linear1.weight: copying a param with shape torch.Size([2048, 256]) from checkpoint, the shape in current model is torch.Size([1024, 256]).
	size mismatch for transformer.encoder.layers.1.linear1.bias: copying a param with shape torch.Size([2048]) from checkpoint, the shape in current model is torch.Size([1024]).
	size mismatch for trans

In [94]:
state_dict['model'].keys()

odict_keys(['transformer.encoder.layers.0.self_attn.in_proj_weight', 'transformer.encoder.layers.0.self_attn.in_proj_bias', 'transformer.encoder.layers.0.self_attn.out_proj.weight', 'transformer.encoder.layers.0.self_attn.out_proj.bias', 'transformer.encoder.layers.0.linear1.weight', 'transformer.encoder.layers.0.linear1.bias', 'transformer.encoder.layers.0.linear2.weight', 'transformer.encoder.layers.0.linear2.bias', 'transformer.encoder.layers.0.norm1.weight', 'transformer.encoder.layers.0.norm1.bias', 'transformer.encoder.layers.0.norm2.weight', 'transformer.encoder.layers.0.norm2.bias', 'transformer.encoder.layers.1.self_attn.in_proj_weight', 'transformer.encoder.layers.1.self_attn.in_proj_bias', 'transformer.encoder.layers.1.self_attn.out_proj.weight', 'transformer.encoder.layers.1.self_attn.out_proj.bias', 'transformer.encoder.layers.1.linear1.weight', 'transformer.encoder.layers.1.linear1.bias', 'transformer.encoder.layers.1.linear2.weight', 'transformer.encoder.layers.1.linear2

In [11]:
import util.misc as utils
from datasets.open_world_eval import OWEvaluator
from datasets.panoptic_eval import PanopticEvaluator
from datasets.data_prefetcher import data_prefetcher

In [ ]:
model.eval()
criterion.eval()
metric_logger = utils.MetricLogger(delimiter="  ")
header = 'Test:'
iou_types = tuple(k for k in ('segm', 'bbox') if k in postprocessors.keys())
coco_evaluator = OWEvaluator(base_ds, iou_types, args=args)

In [ ]:
for samples, targets in metric_logger.log_every(data_loader, 10, header):
    samples = samples.to(device)
    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
    outputs = model(samples)
    orig_target_sizes = torch.stack([t["orig_size"] for t in targets], dim=0)
    results = postprocessors['bbox'](outputs, orig_target_sizes)

## Update XML files
Also in script `update_xml.py` - still needs a main.

### Add missing tags to XML

In [2]:
import os
import re

def replace_in_file(file_path, pattern, replacement):
    with open(file_path, 'r+') as file:
        file_content = file.read()
        file_content = re.sub(pattern, replacement, file_content)
        file.seek(0)
        file.write(file_content)
        file.truncate()

def replace_in_all_files(directory, pattern, replacement):
    for foldername, subfolders, filenames in os.walk(directory):
        for filename in filenames:
            file_path = os.path.join(foldername, filename)
            replace_in_file(file_path, pattern, replacement)

directory = "./data/OWOD/Annotations/"
pattern = "/name>\n        <bndbox>"
replacement = "/name>\n        <truncated>0</truncated>\n        <difficult>0</difficult>\n        <bndbox>"


In [ ]:
# replace_in_all_files(directory, pattern, replacement)

### Update annotation fiel with new path and new name.

In [73]:
import pandas as pd
import os
import xml.etree.ElementTree as ET
import logging
from tqdm import tqdm

dict_csv = "./data/OWOD/filename_map.csv"
xml_dir = "./data/OWOD/Annotations/"
new_path_prefix = "/home/sabrina/code/PROB/data/OWOD/ImageSets/"

def main(dict_csv, xml_dir, new_path_prefix):
    logging.basicConfig(filename='xml_update.log', level=logging.INFO)
    
    df = pd.read_csv(dict_csv)
    name_dict = df.set_index('old_name')['new_name'].to_dict()

    xml_files = [f for f in os.listdir(xml_dir) if f.endswith('.xml')]

    for xml_file in tqdm(xml_files, desc="Updating XML files"):
        tree = ET.parse(os.path.join(xml_dir, xml_file))
        root = tree.getroot()

        for elem in root.iter():
            try:
                if elem.tag == 'filename':
                    old_filename = elem.text.split('.')[0]
                    file_extension = elem.text.split('.')[1]

                    new_filename = f"{name_dict[old_filename]}.{file_extension}"
                    new_path = f"{new_path_prefix}{new_filename}"

                    elem.text = new_filename

                if elem.tag == 'path':
                    elem.text = new_path

            except Exception as e:
                logging.error(f"Error processing XML file {xml_file}: {e}")
                pass

        tree.write(os.path.join(xml_dir, xml_file))                    


In [75]:
# main(dict_csv, xml_dir, new_path_prefix)

: 

In [17]:
!python -u main_open_world.py \ 
--eval \
--pretrain "/exps/MOWODB/PROB/t1/checkpoint0040.pth"

SyntaxError: invalid syntax (480740770.py, line 3)